# Dynamic Programming

*Solving complex problems by breaking them into overlapping subproblems*


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⏱️ OTURUM ZAMANLAYICI — Bu hücreyi ilk çalıştırın!
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import time as _time, datetime as _dt, json as _json, os as _os
from IPython.display import display, HTML as _HTML

# ── Persistent tracker file (survives kernel restarts in same runtime) ──
_TRACKER_FILE = _os.path.join(
    '/content' if _os.path.isdir('/content') else '/tmp',
    '.cp2_session_Week_12.json'
)

def _load_tracker():
    """Load previous sessions from file."""
    try:
        with open(_TRACKER_FILE, 'r') as f:
            return _json.load(f)
    except (FileNotFoundError, _json.JSONDecodeError, ValueError):
        return {'sessions': [], 'total_heartbeats': 0}

def _save_tracker(data):
    """Persist tracker data to file."""
    try:
        with open(_TRACKER_FILE, 'w') as f:
            _json.dump(data, f)
    except OSError:
        pass

# ── Load any previous session data ──
_tracker = _load_tracker()
_prev_sessions = len(_tracker['sessions'])
_prev_hb = _tracker['total_heartbeats']

# ── Start new session ──
_SESSION_START = _time.time()
_SESSION_START_STR = _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
_HEARTBEATS = [_time.time()]
_CELLS_RUN = [0]
IDLE_THRESHOLD = 300   # 5 min

# Record this session start
_tracker['sessions'].append({
    'start': _SESSION_START_STR,
    'start_ts': _SESSION_START,
    'heartbeats': [_SESSION_START]
})
_save_tracker(_tracker)

def _heartbeat_hook(*args, **kwargs):
    """Record each cell execution — in-memory + file."""
    now = _time.time()
    _HEARTBEATS.append(now)
    _CELLS_RUN[0] += 1
    # Persist every 5 cells to avoid excessive disk IO
    if _CELLS_RUN[0] % 5 == 0:
        try:
            _t = _load_tracker()
            if _t['sessions']:
                _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
                _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
                _t['total_heartbeats'] = _prev_hb + len(_HEARTBEATS)
                _save_tracker(_t)
        except Exception:
            pass

def _calc_active_time(heartbeats=None):
    """Calculate active time from heartbeat list, capping idle gaps."""
    hb = heartbeats or _HEARTBEATS
    if len(hb) < 2: return 0
    active = 0
    for i in range(1, len(hb)):
        gap = hb[i] - hb[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _calc_total_active_time():
    """Calculate total across ALL sessions (multi-day support)."""
    total = 0
    try:
        _t = _load_tracker()
        for sess in _t['sessions'][:-1]:  # all previous sessions
            hb = sess.get('heartbeats', [])
            if len(hb) >= 2:
                for i in range(1, len(hb)):
                    gap = hb[i] - hb[i-1]
                    total += gap if gap <= IDLE_THRESHOLD else 30
    except Exception:
        pass
    # Add current session
    total += _calc_active_time()
    return int(total)

try:
    _ip = get_ipython()
    _ip.events.register('pre_run_cell', _heartbeat_hook)
except Exception: pass

# ── Display ──
_multi = ''
if _prev_sessions > 0:
    _multi = f'<br><span style="font-size:12px;opacity:.75">📂 {_prev_sessions} önceki oturum bulundu — çoklu oturum takibi aktif.</span>'
display(_HTML(f"""<div style='background:linear-gradient(135deg,#667eea,#764ba2);padding:14px 20px;border-radius:10px;color:white;font-family:system-ui;margin:4px 0'>
<b>⏱️ Oturum Başladı</b> — {_SESSION_START_STR}<br>
<span style='font-size:13px;opacity:.85'>Hücre aktiviteniz takip ediliyor. Bitince en alttaki Submit hücresini çalıştırın.</span>{_multi}</div>"""))
print(f'✅ Zamanlayıcı aktif. Idle eşiği: {IDLE_THRESHOLD//60} dk')
if _prev_sessions > 0:
    print(f'📂 Önceki oturumlar: {_prev_sessions} | Toplam heartbeat: {_prev_hb}')

## 🎯 Learning Objectives

- Understand the dynamic programming paradigm: optimal substructure and overlapping subproblems
- Implement DP solutions using both top-down (memoization) and bottom-up (tabulation) approaches
- Solve classic 1D DP problems: climbing stairs, house robber, coin change, longest increasing subsequence
- Solve 2D DP problems: unique paths, edit distance, longest common subsequence, 0/1 knapsack
- Apply space optimization techniques to reduce DP memory usage from O(n²) to O(n)

---
## 1. Introduction to Dynamic Programming


Dynamic Programming (DP) is a powerful algorithmic technique for solving problems by breaking them into smaller overlapping subproblems. Instead of solving the same subproblem multiple times, DP stores results and reuses them. This transforms exponential time complexity into polynomial time for many problems.

> 📖 **Definition:** **Dynamic Programming** applies when a problem has two key properties: **optimal substructure** (optimal solution contains optimal solutions to subproblems) and **overlapping subproblems** (same subproblems are solved repeatedly). DP trades space for time by caching computed results.

### DP vs Other Techniques

| Technique | Subproblems | Approach | Example |
| --- | --- | --- | --- |
| Divide & Conquer | Non-overlapping | Split, solve, combine | Merge Sort |
| Greedy | Local optimal | Make best choice now | Huffman Coding |
| Dynamic Programming | Overlapping | Cache & reuse | Fibonacci, Knapsack |
| Backtracking | Decision tree | Try all, prune | N-Queens |

**Listing 12.1 — Fibonacci: Recursion vs DP**

In [ ]:
import time

def fib_recursive(n):
    """Naive recursion - O(2^n) time."""
    if n <= 1:
        return n
    return fib_recursive(n - 1) + fib_recursive(n - 2)

def fib_dp(n):
    """DP with memoization - O(n) time."""
    memo = {}
    def helper(n):
        if n in memo:
            return memo[n]
        if n <= 1:
            return n
        memo[n] = helper(n - 1) + helper(n - 2)
        return memo[n]
    return helper(n)

# Compare performance
n = 30
start = time.perf_counter()
result1 = fib_recursive(n)
time1 = time.perf_counter() - start

start = time.perf_counter()
result2 = fib_dp(n)
time2 = time.perf_counter() - start

print(f"Fibonacci({n}) = {result1}")
print(f"Recursive: {time1*1000:.2f} ms")
print(f"DP:        {time2*1000:.4f} ms")
print(f"Speedup:   {time1/time2:.0f}x")

***Figure 12.1:** DP eliminates redundant calculations—fib(30) goes from millions of calls to just 30.*

---
## 2. Memoization (Top-Down)


> 📖 **Definition:** **Memoization** is the top-down approach to DP. Start with the original problem, recursively break it down, and cache results as you compute them. Think: "solve by recursion, but remember answers."

**Listing 12.2 — Memoization with Dictionary**

In [ ]:
def climb_stairs_memo(n):
    """
    Count ways to climb n stairs (1 or 2 steps at a time).
    Top-down memoization approach.
    """
    memo = {}
    
    def helper(n):
        if n in memo:
            return memo[n]
        if n <= 2:
            return n
        
        memo[n] = helper(n - 1) + helper(n - 2)
        return memo[n]
    
    return helper(n)

# Trace the memoization
def climb_stairs_traced(n, memo=None, depth=0):
    if memo is None:
        memo = {}
    
    indent = "  " * depth
    if n in memo:
        print(f"{indent}climb({n}) -> cached: {memo[n]}")
        return memo[n]
    
    print(f"{indent}climb({n}) -> computing...")
    if n <= 2:
        memo[n] = n
    else:
        memo[n] = climb_stairs_traced(n-1, memo, depth+1) + \
                  climb_stairs_traced(n-2, memo, depth+1)
    
    print(f"{indent}climb({n}) = {memo[n]}")
    return memo[n]

print("Tracing climb_stairs(5):")
result = climb_stairs_traced(5)
print(f"\nWays to climb 5 stairs: {result}")

***Figure 12.2:** Memoization caches results as computed—subsequent calls return instantly.*

**Listing 12.3 — Using functools.lru_cache**

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib(n):
    """Fibonacci with automatic memoization."""
    if n <= 1:
        return n
    return fib(n - 1) + fib(n - 2)

# Test
print(f"fib(50) = {fib(50)}")

# Check cache statistics
print(f"\nCache info: {fib.cache_info()}")

# Clear cache if needed
# fib.cache_clear()

@lru_cache(maxsize=None)
def unique_paths(m, n):
    """Count paths in m x n grid (only right or down)."""
    if m == 1 or n == 1:
        return 1
    return unique_paths(m - 1, n) + unique_paths(m, n - 1)

print(f"\nUnique paths in 3x7 grid: {unique_paths(3, 7)}")

***Figure 12.3:** @lru_cache provides automatic memoization—cleanest way in Python.*

---
## 3. Tabulation (Bottom-Up)


> 📖 **Definition:** **Tabulation** is the bottom-up approach to DP. Start with the smallest subproblems, solve them first, and build up to the answer. Uses iteration instead of recursion, avoiding stack overhead.

**Listing 12.4 — Tabulation for Fibonacci**

In [ ]:
def fib_tabulation(n):
    """
    Fibonacci using bottom-up tabulation.
    O(n) time, O(n) space.
    """
    if n <= 1:
        return n
    
    # Create table and fill base cases
    dp = [0] * (n + 1)
    dp[0] = 0
    dp[1] = 1
    
    # Fill table bottom-up
    for i in range(2, n + 1):
        dp[i] = dp[i - 1] + dp[i - 2]
    
    return dp[n]

# Trace the filling process
def fib_tabulation_traced(n):
    if n <= 1:
        return n
    
    dp = [0] * (n + 1)
    dp[0], dp[1] = 0, 1
    
    print("Building table bottom-up:")
    print(f"  dp[0] = 0 (base)")
    print(f"  dp[1] = 1 (base)")
    
    for i in range(2, n + 1):
        dp[i] = dp[i - 1] + dp[i - 2]
        print(f"  dp[{i}] = dp[{i-1}] + dp[{i-2}] = {dp[i-1]} + {dp[i-2]} = {dp[i]}")
    
    return dp[n]

result = fib_tabulation_traced(8)
print(f"\nFibonacci(8) = {result}")

***Figure 12.4:** Tabulation fills the DP table iteratively from base cases to answer.*

**Listing 12.5 — Tabulation for Climbing Stairs**

In [ ]:
def climb_stairs_tabulation(n):
    """
    Count ways to climb n stairs using tabulation.
    dp[i] = number of ways to reach step i
    """
    if n <= 2:
        return n
    
    dp = [0] * (n + 1)
    dp[1] = 1  # One way to reach step 1
    dp[2] = 2  # Two ways to reach step 2
    
    for i in range(3, n + 1):
        dp[i] = dp[i - 1] + dp[i - 2]
    
    return dp[n]

# Test
for n in range(1, 8):
    print(f"Ways to climb {n} stairs: {climb_stairs_tabulation(n)}")

# Visualize the recurrence relation
print("\nRecurrence: dp[i] = dp[i-1] + dp[i-2]")
print("From step i-1: take 1 step")
print("From step i-2: take 2 steps")

***Figure 12.5:** Each state depends on previous states—the recurrence relation.*

---
## 4. 1D Dynamic Programming


**Listing 12.6 — House Robber**

In [ ]:
def rob(nums):
    """
    Maximum money robbing non-adjacent houses.
    dp[i] = max money robbing houses 0 to i
    Choice: rob house i or skip it
    """
    if not nums:
        return 0
    if len(nums) == 1:
        return nums[0]
    
    n = len(nums)
    dp = [0] * n
    dp[0] = nums[0]
    dp[1] = max(nums[0], nums[1])
    
    for i in range(2, n):
        # Either rob house i (add to dp[i-2])
        # Or skip house i (keep dp[i-1])
        dp[i] = max(dp[i - 1], dp[i - 2] + nums[i])
    
    return dp[-1]

# Test
houses = [2, 7, 9, 3, 1]
print(f"Houses: {houses}")
print(f"Max profit: {rob(houses)}")  # Rob houses 0, 2, 4: 2+9+1=12

houses = [1, 2, 3, 1]
print(f"\nHouses: {houses}")
print(f"Max profit: {rob(houses)}")  # Rob houses 0, 2: 1+3=4

***Figure 12.6:** At each house, decide whether to rob it (skip previous) or skip it.*

**Listing 12.7 — Maximum Subarray (Kadane's)**

In [ ]:
def max_subarray(nums):
    """
    Find contiguous subarray with largest sum (Kadane's algorithm).
    dp[i] = max sum of subarray ending at index i
    """
    n = len(nums)
    dp = [0] * n
    dp[0] = nums[0]
    
    for i in range(1, n):
        # Either extend previous subarray or start new one
        dp[i] = max(dp[i - 1] + nums[i], nums[i])
    
    return max(dp)

# Space-optimized version
def max_subarray_optimized(nums):
    """O(1) space version."""
    max_sum = current_sum = nums[0]
    
    for num in nums[1:]:
        current_sum = max(current_sum + num, num)
        max_sum = max(max_sum, current_sum)
    
    return max_sum

# Test
nums = [-2, 1, -3, 4, -1, 2, 1, -5, 4]
print(f"Array: {nums}")
print(f"Max subarray sum: {max_subarray(nums)}")  # [4,-1,2,1] = 6
print(f"Optimized result: {max_subarray_optimized(nums)}")

***Figure 12.7:** Kadane's algorithm: extend or restart at each element.*

**Listing 12.8 — Coin Change (Minimum Coins)**

In [ ]:
def coin_change(coins, amount):
    """
    Minimum coins to make amount.
    dp[i] = minimum coins needed for amount i
    """
    dp = [float('inf')] * (amount + 1)
    dp[0] = 0  # 0 coins for amount 0
    
    for i in range(1, amount + 1):
        for coin in coins:
            if coin <= i and dp[i - coin] != float('inf'):
                dp[i] = min(dp[i], dp[i - coin] + 1)
    
    return dp[amount] if dp[amount] != float('inf') else -1

# Test
coins = [1, 2, 5]
amount = 11
print(f"Coins: {coins}, Amount: {amount}")
print(f"Minimum coins: {coin_change(coins, amount)}")  # 5+5+1 = 3 coins

# Trace for smaller example
coins = [1, 2, 5]
amount = 7
print(f"\nBuilding dp for amount {amount}:")
dp = [float('inf')] * (amount + 1)
dp[0] = 0
for i in range(1, amount + 1):
    for coin in coins:
        if coin <= i and dp[i - coin] != float('inf'):
            dp[i] = min(dp[i], dp[i - coin] + 1)
    print(f"  dp[{i}] = {dp[i]}")

***Figure 12.8:** For each amount, try all coins and take minimum.*

---
## 5. 2D Dynamic Programming


**Listing 12.9 — Unique Paths**

In [ ]:
def unique_paths(m, n):
    """
    Count paths from top-left to bottom-right in m x n grid.
    Only move right or down.
    dp[i][j] = number of paths to reach cell (i, j)
    """
    dp = [[1] * n for _ in range(m)]
    
    # First row and column are all 1s (only one way to reach)
    # Fill rest of the grid
    for i in range(1, m):
        for j in range(1, n):
            dp[i][j] = dp[i - 1][j] + dp[i][j - 1]
    
    return dp[m - 1][n - 1]

# Test and visualize
m, n = 3, 4
print(f"Grid: {m} x {n}")
dp = [[1] * n for _ in range(m)]
for i in range(1, m):
    for j in range(1, n):
        dp[i][j] = dp[i - 1][j] + dp[i][j - 1]

print("DP table:")
for row in dp:
    print(f"  {row}")
print(f"\nUnique paths: {dp[m-1][n-1]}")

***Figure 12.9:** Each cell sums paths from above and left.*

**Listing 12.10 — Minimum Path Sum**

In [ ]:
def min_path_sum(grid):
    """
    Find path from top-left to bottom-right with minimum sum.
    dp[i][j] = minimum sum to reach cell (i, j)
    """
    m, n = len(grid), len(grid[0])
    dp = [[0] * n for _ in range(m)]
    
    # Base case
    dp[0][0] = grid[0][0]
    
    # First row (can only come from left)
    for j in range(1, n):
        dp[0][j] = dp[0][j - 1] + grid[0][j]
    
    # First column (can only come from above)
    for i in range(1, m):
        dp[i][0] = dp[i - 1][0] + grid[i][0]
    
    # Fill rest
    for i in range(1, m):
        for j in range(1, n):
            dp[i][j] = min(dp[i - 1][j], dp[i][j - 1]) + grid[i][j]
    
    return dp[m - 1][n - 1]

# Test
grid = [
    [1, 3, 1],
    [1, 5, 1],
    [4, 2, 1]
]
print("Grid:")
for row in grid:
    print(f"  {row}")
print(f"\nMinimum path sum: {min_path_sum(grid)}")  # 1→3→1→1→1 = 7

***Figure 12.10:** Take minimum of paths from above and left, add current cell.*

**Listing 12.11 — Unique Paths with Obstacles**

In [ ]:
def unique_paths_with_obstacles(grid):
    """
    Count paths avoiding obstacles (marked as 1).
    dp[i][j] = 0 if obstacle, else sum of paths from above and left
    """
    m, n = len(grid), len(grid[0])
    
    # If start or end is blocked
    if grid[0][0] == 1 or grid[m-1][n-1] == 1:
        return 0
    
    dp = [[0] * n for _ in range(m)]
    dp[0][0] = 1
    
    # First row
    for j in range(1, n):
        dp[0][j] = dp[0][j - 1] if grid[0][j] == 0 else 0
    
    # First column
    for i in range(1, m):
        dp[i][0] = dp[i - 1][0] if grid[i][0] == 0 else 0
    
    # Fill rest
    for i in range(1, m):
        for j in range(1, n):
            if grid[i][j] == 0:
                dp[i][j] = dp[i - 1][j] + dp[i][j - 1]
            # else dp[i][j] stays 0
    
    return dp[m - 1][n - 1]

# Test
grid = [
    [0, 0, 0],
    [0, 1, 0],
    [0, 0, 0]
]
print("Grid (1 = obstacle):")
for row in grid:
    print(f"  {row}")
print(f"\nUnique paths: {unique_paths_with_obstacles(grid)}")

***Figure 12.11:** Obstacles propagate zeros—no paths through blocked cells.*

---
## 6. Subsequence Problems


**Listing 12.12 — Longest Increasing Subsequence**

In [ ]:
def length_of_lis(nums):
    """
    Find length of longest strictly increasing subsequence.
    dp[i] = length of LIS ending at index i
    O(n²) time
    """
    n = len(nums)
    dp = [1] * n  # Every element is LIS of length 1
    
    for i in range(1, n):
        for j in range(i):
            if nums[j] < nums[i]:
                dp[i] = max(dp[i], dp[j] + 1)
    
    return max(dp)

# O(n log n) solution using binary search
import bisect

def length_of_lis_optimized(nums):
    """O(n log n) using patience sorting."""
    tails = []  # tails[i] = smallest tail of LIS of length i+1
    
    for num in nums:
        pos = bisect.bisect_left(tails, num)
        if pos == len(tails):
            tails.append(num)
        else:
            tails[pos] = num
    
    return len(tails)

# Test
nums = [10, 9, 2, 5, 3, 7, 101, 18]
print(f"Array: {nums}")
print(f"LIS length (O(n²)): {length_of_lis(nums)}")
print(f"LIS length (O(n log n)): {length_of_lis_optimized(nums)}")
# LIS: [2, 3, 7, 101] or [2, 5, 7, 101]

***Figure 12.12:** LIS at each position considers all smaller previous elements.*

**Listing 12.13 — Longest Common Subsequence**

In [ ]:
def longest_common_subsequence(text1, text2):
    """
    Find length of LCS of two strings.
    dp[i][j] = LCS length of text1[:i] and text2[:j]
    """
    m, n = len(text1), len(text2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if text1[i - 1] == text2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])
    
    return dp[m][n]

# Reconstruct the LCS
def get_lcs(text1, text2):
    m, n = len(text1), len(text2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if text1[i - 1] == text2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])
    
    # Backtrack to find LCS
    lcs = []
    i, j = m, n
    while i > 0 and j > 0:
        if text1[i - 1] == text2[j - 1]:
            lcs.append(text1[i - 1])
            i -= 1
            j -= 1
        elif dp[i - 1][j] > dp[i][j - 1]:
            i -= 1
        else:
            j -= 1
    
    return ''.join(reversed(lcs))

# Test
text1, text2 = "abcde", "ace"
print(f"text1: {text1}")
print(f"text2: {text2}")
print(f"LCS length: {longest_common_subsequence(text1, text2)}")
print(f"LCS: {get_lcs(text1, text2)}")

***Figure 12.13:** If characters match, extend LCS; otherwise take max of excluding either.*

**Listing 12.14 — Longest Palindromic Subsequence**

In [ ]:
def longest_palindrome_subseq(s):
    """
    Find length of longest palindromic subsequence.
    Key insight: LPS(s) = LCS(s, reverse(s))
    Or use interval DP: dp[i][j] = LPS of s[i:j+1]
    """
    n = len(s)
    dp = [[0] * n for _ in range(n)]
    
    # Base case: single characters
    for i in range(n):
        dp[i][i] = 1
    
    # Fill for increasing lengths
    for length in range(2, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            if s[i] == s[j]:
                dp[i][j] = dp[i + 1][j - 1] + 2
            else:
                dp[i][j] = max(dp[i + 1][j], dp[i][j - 1])
    
    return dp[0][n - 1]

# Test
s = "bbbab"
print(f"String: {s}")
print(f"Longest palindromic subsequence length: {longest_palindrome_subseq(s)}")
# LPS: "bbbb" = 4

s = "cbbd"
print(f"\nString: {s}")
print(f"Longest palindromic subsequence length: {longest_palindrome_subseq(s)}")
# LPS: "bb" = 2

***Figure 12.14:** Interval DP: expand from single characters to full string.*

---
## 7. Knapsack Problems


**Listing 12.15 — 0/1 Knapsack**

In [ ]:
def knapsack_01(weights, values, capacity):
    """
    0/1 Knapsack: each item used at most once.
    dp[i][w] = max value using items 0..i-1 with capacity w
    """
    n = len(weights)
    dp = [[0] * (capacity + 1) for _ in range(n + 1)]
    
    for i in range(1, n + 1):
        for w in range(capacity + 1):
            # Don't take item i-1
            dp[i][w] = dp[i - 1][w]
            
            # Take item i-1 if it fits
            if weights[i - 1] <= w:
                dp[i][w] = max(dp[i][w], 
                              dp[i - 1][w - weights[i - 1]] + values[i - 1])
    
    return dp[n][capacity]

# Test
weights = [1, 2, 3, 4]
values = [10, 20, 30, 40]
capacity = 5
print(f"Weights: {weights}")
print(f"Values:  {values}")
print(f"Capacity: {capacity}")
print(f"Max value: {knapsack_01(weights, values, capacity)}")
# Take items with weights 2 and 3: values 20 + 30 = 50

***Figure 12.15:** For each item, decide to take it or not based on remaining capacity.*

**Listing 12.16 — Unbounded Knapsack**

In [ ]:
def knapsack_unbounded(weights, values, capacity):
    """
    Unbounded Knapsack: each item can be used unlimited times.
    dp[w] = max value achievable with capacity w
    """
    dp = [0] * (capacity + 1)
    
    for w in range(1, capacity + 1):
        for i in range(len(weights)):
            if weights[i] <= w:
                dp[w] = max(dp[w], dp[w - weights[i]] + values[i])
    
    return dp[capacity]

# Test
weights = [1, 3, 4]
values = [10, 40, 50]
capacity = 8
print(f"Weights: {weights}")
print(f"Values:  {values}")
print(f"Capacity: {capacity}")
print(f"Max value (unbounded): {knapsack_unbounded(weights, values, capacity)}")
# Take item 1 twice (weight 3+3=6, value 40+40=80), then item 0 twice (weight 1+1=2, value 10+10=20)

***Figure 12.16:** Unbounded allows reusing items—check all items for each capacity.*

**Listing 12.17 — Coin Change II (Count Ways)**

In [ ]:
def change(amount, coins):
    """
    Count number of ways to make amount using coins.
    Each coin can be used unlimited times.
    dp[i] = number of ways to make amount i
    """
    dp = [0] * (amount + 1)
    dp[0] = 1  # One way to make 0: use no coins
    
    # Process coins one by one to avoid counting permutations
    for coin in coins:
        for i in range(coin, amount + 1):
            dp[i] += dp[i - coin]
    
    return dp[amount]

# Test
coins = [1, 2, 5]
amount = 5
print(f"Coins: {coins}, Amount: {amount}")
print(f"Number of ways: {change(amount, coins)}")
# Ways: 5=5, 5=2+2+1, 5=2+1+1+1, 5=1+1+1+1+1 → 4 ways

# Trace for smaller example
coins = [1, 2]
amount = 4
print(f"\nCoins: {coins}, Amount: {amount}")
dp = [0] * (amount + 1)
dp[0] = 1
for coin in coins:
    for i in range(coin, amount + 1):
        dp[i] += dp[i - coin]
    print(f"After coin {coin}: {dp}")
print(f"Ways: {dp[amount]}")

***Figure 12.17:** Process coins in outer loop to count combinations, not permutations.*

**Listing 12.18 — Partition Equal Subset Sum**

In [ ]:
def can_partition(nums):
    """
    Can array be partitioned into two subsets with equal sum?
    Equivalent to: can we find subset with sum = total/2?
    This is a 0/1 knapsack variant.
    """
    total = sum(nums)
    
    # If total is odd, impossible to split equally
    if total % 2 != 0:
        return False
    
    target = total // 2
    dp = [False] * (target + 1)
    dp[0] = True  # Sum of 0 is achievable
    
    for num in nums:
        # Traverse backwards to avoid using same number twice
        for j in range(target, num - 1, -1):
            dp[j] = dp[j] or dp[j - num]
    
    return dp[target]

# Test
nums = [1, 5, 11, 5]
print(f"Array: {nums}")
print(f"Can partition equally: {can_partition(nums)}")
# [1, 5, 5] and [11], both sum to 11

nums = [1, 2, 3, 5]
print(f"\nArray: {nums}")
print(f"Can partition equally: {can_partition(nums)}")
# Total = 11 (odd), impossible

***Figure 12.18:** Partition = knapsack with target = sum/2. Traverse backwards for 0/1.*

---
## 8. String DP


**Listing 12.19 — Edit Distance**

In [ ]:
def min_distance(word1, word2):
    """
    Minimum operations (insert, delete, replace) to convert word1 to word2.
    dp[i][j] = edit distance between word1[:i] and word2[:j]
    """
    m, n = len(word1), len(word2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    # Base cases: convert to/from empty string
    for i in range(m + 1):
        dp[i][0] = i  # Delete all chars
    for j in range(n + 1):
        dp[0][j] = j  # Insert all chars
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if word1[i - 1] == word2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]  # No operation needed
            else:
                dp[i][j] = 1 + min(
                    dp[i - 1][j],      # Delete from word1
                    dp[i][j - 1],      # Insert into word1
                    dp[i - 1][j - 1]   # Replace
                )
    
    return dp[m][n]

# Test
word1, word2 = "horse", "ros"
print(f"word1: {word1}")
print(f"word2: {word2}")
print(f"Edit distance: {min_distance(word1, word2)}")
# horse -> rorse (replace h with r)
# rorse -> rose (delete r)
# rose -> ros (delete e)

***Figure 12.19:** At each position, choose minimum of insert, delete, or replace.*

**Listing 12.20 — Longest Palindromic Substring**

In [ ]:
def longest_palindrome(s):
    """
    Find longest palindromic substring.
    dp[i][j] = True if s[i:j+1] is palindrome
    """
    n = len(s)
    if n < 2:
        return s
    
    dp = [[False] * n for _ in range(n)]
    start, max_len = 0, 1
    
    # All single chars are palindromes
    for i in range(n):
        dp[i][i] = True
    
    # Check length 2
    for i in range(n - 1):
        if s[i] == s[i + 1]:
            dp[i][i + 1] = True
            start, max_len = i, 2
    
    # Check lengths 3 to n
    for length in range(3, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            if s[i] == s[j] and dp[i + 1][j - 1]:
                dp[i][j] = True
                start, max_len = i, length
    
    return s[start:start + max_len]

# Test
s = "babad"
print(f"String: {s}")
print(f"Longest palindromic substring: {longest_palindrome(s)}")

s = "cbbd"
print(f"\nString: {s}")
print(f"Longest palindromic substring: {longest_palindrome(s)}")

***Figure 12.20:** A substring is palindrome if ends match and middle is palindrome.*

**Listing 12.21 — Word Break**

In [ ]:
def word_break(s, word_dict):
    """
    Can string be segmented into dictionary words?
    dp[i] = True if s[:i] can be segmented
    """
    word_set = set(word_dict)
    n = len(s)
    dp = [False] * (n + 1)
    dp[0] = True  # Empty string is valid
    
    for i in range(1, n + 1):
        for j in range(i):
            if dp[j] and s[j:i] in word_set:
                dp[i] = True
                break
    
    return dp[n]

# Test
s = "leetcode"
word_dict = ["leet", "code"]
print(f"String: {s}")
print(f"Dictionary: {word_dict}")
print(f"Can break: {word_break(s, word_dict)}")

s = "applepenapple"
word_dict = ["apple", "pen"]
print(f"\nString: {s}")
print(f"Dictionary: {word_dict}")
print(f"Can break: {word_break(s, word_dict)}")

***Figure 12.21:** Check if any valid split point leads to solution.*

---
## 9. Space Optimization


**Listing 12.22 — Fibonacci O(1) Space**

In [ ]:
def fib_optimized(n):
    """
    Fibonacci with O(1) space.
    Only need previous two values, not entire array.
    """
    if n <= 1:
        return n
    
    prev2, prev1 = 0, 1
    
    for _ in range(2, n + 1):
        current = prev1 + prev2
        prev2, prev1 = prev1, current
    
    return prev1

# Compare space usage
def fib_o_n_space(n):
    """O(n) space version for comparison."""
    if n <= 1:
        return n
    dp = [0] * (n + 1)
    dp[1] = 1
    for i in range(2, n + 1):
        dp[i] = dp[i - 1] + dp[i - 2]
    return dp[n]

n = 50
print(f"fib({n}) = {fib_optimized(n)}")
print(f"O(n) space uses {n+1} integers")
print(f"O(1) space uses 2 integers")

***Figure 12.22:** When each state only depends on previous few, keep only those.*

**Listing 12.23 — 2D to 1D Space Reduction**

In [ ]:
def unique_paths_1d(m, n):
    """
    Unique paths using O(n) space instead of O(m*n).
    Since we only need previous row, keep one row.
    """
    dp = [1] * n  # Start with first row (all 1s)
    
    for _ in range(1, m):
        for j in range(1, n):
            # dp[j] = dp[j] (above) + dp[j-1] (left)
            # dp[j] already contains value from previous row
            dp[j] += dp[j - 1]
    
    return dp[n - 1]

# Test
m, n = 3, 7
print(f"Grid: {m} x {n}")
print(f"Unique paths (O(n) space): {unique_paths_1d(m, n)}")

# Trace
m, n = 3, 4
dp = [1] * n
print(f"\nTracing {m}x{n} grid:")
print(f"Initial row: {dp}")
for i in range(1, m):
    for j in range(1, n):
        dp[j] += dp[j - 1]
    print(f"After row {i}: {dp}")
print(f"Result: {dp[n-1]}")

***Figure 12.23:** Reuse same array—dp[j] holds "above" until we update it.*

**Listing 12.24 — LCS with O(n) Space**

In [ ]:
def lcs_optimized(text1, text2):
    """
    LCS with O(min(m,n)) space.
    Only keep two rows (or one row with care).
    """
    # Ensure text2 is shorter for less space
    if len(text1) < len(text2):
        text1, text2 = text2, text1
    
    m, n = len(text1), len(text2)
    prev = [0] * (n + 1)
    curr = [0] * (n + 1)
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if text1[i - 1] == text2[j - 1]:
                curr[j] = prev[j - 1] + 1
            else:
                curr[j] = max(prev[j], curr[j - 1])
        prev, curr = curr, prev  # Swap rows
    
    return prev[n]

# Test
text1 = "abcde"
text2 = "ace"
print(f"text1: {text1}")
print(f"text2: {text2}")
print(f"LCS length: {lcs_optimized(text1, text2)}")

print(f"\nSpace: O({min(len(text1), len(text2))}) instead of O({len(text1)*len(text2)})")

***Figure 12.24:** Two-row technique reduces m×n space to 2×min(m,n).*

---
## 10. Common Patterns


### Pattern 1: Linear DP

**Listing 12.25 — Decode Ways**

In [ ]:
def num_decodings(s):
    """
    Count ways to decode string where A=1, B=2, ..., Z=26.
    dp[i] = ways to decode s[:i]
    """
    if not s or s[0] == '0':
        return 0
    
    n = len(s)
    dp = [0] * (n + 1)
    dp[0] = 1  # Empty string
    dp[1] = 1  # First char (already checked not '0')
    
    for i in range(2, n + 1):
        # Single digit (1-9)
        if s[i - 1] != '0':
            dp[i] += dp[i - 1]
        
        # Two digits (10-26)
        two_digit = int(s[i - 2:i])
        if 10 <= two_digit <= 26:
            dp[i] += dp[i - 2]
    
    return dp[n]

# Test
s = "226"
print(f"String: {s}")
print(f"Decode ways: {num_decodings(s)}")
# "BZ" (2 26), "VF" (22 6), "BBF" (2 2 6) → 3 ways

s = "12"
print(f"\nString: {s}")
print(f"Decode ways: {num_decodings(s)}")
# "AB" (1 2), "L" (12) → 2 ways

***Figure 12.25:** At each position, consider taking 1 or 2 digits if valid.*

### Pattern 2: Interval DP

**Listing 12.26 — Burst Balloons**

In [ ]:
def max_coins(nums):
    """
    Maximum coins from bursting all balloons.
    dp[i][j] = max coins bursting balloons between i and j (exclusive).
    Key insight: think of balloon k as LAST to burst in interval.
    """
    # Add boundary balloons
    nums = [1] + nums + [1]
    n = len(nums)
    dp = [[0] * n for _ in range(n)]
    
    # For each interval length
    for length in range(2, n):
        for left in range(n - length):
            right = left + length
            # Try each balloon as last to burst
            for k in range(left + 1, right):
                coins = nums[left] * nums[k] * nums[right]
                total = dp[left][k] + coins + dp[k][right]
                dp[left][right] = max(dp[left][right], total)
    
    return dp[0][n - 1]

# Test
nums = [3, 1, 5, 8]
print(f"Balloons: {nums}")
print(f"Max coins: {max_coins(nums)}")
# Burst 1: 3*1*5 = 15, array becomes [3,5,8]
# Burst 5: 3*5*8 = 120, array becomes [3,8]
# Burst 3: 1*3*8 = 24, array becomes [8]
# Burst 8: 1*8*1 = 8
# Total: 15 + 120 + 24 + 8 = 167

***Figure 12.26:** Interval DP: think backwards—which balloon to burst last?*

### Pattern 3: State Machine DP

**Listing 12.27 — Best Time to Buy and Sell Stock with Cooldown**

In [ ]:
def max_profit_cooldown(prices):
    """
    Buy/sell with 1-day cooldown after selling.
    State machine: hold, sold, rest
    """
    if not prices:
        return 0
    
    n = len(prices)
    hold = [0] * n    # Holding stock
    sold = [0] * n    # Just sold
    rest = [0] * n    # Resting (cooldown or doing nothing)
    
    hold[0] = -prices[0]
    sold[0] = 0
    rest[0] = 0
    
    for i in range(1, n):
        hold[i] = max(hold[i-1], rest[i-1] - prices[i])  # Keep or buy
        sold[i] = hold[i-1] + prices[i]                   # Sell
        rest[i] = max(rest[i-1], sold[i-1])               # Keep resting or enter cooldown
    
    return max(sold[n-1], rest[n-1])

# Test
prices = [1, 2, 3, 0, 2]
print(f"Prices: {prices}")
print(f"Max profit with cooldown: {max_profit_cooldown(prices)}")
# Buy at 1, sell at 3, cooldown, buy at 0, sell at 2 → 3

***Figure 12.27:** State machine DP: track multiple states and transitions.*

### Pattern 4: Counting DP

**Listing 12.28 — Perfect Squares**

In [ ]:
def num_squares(n):
    """
    Minimum perfect squares that sum to n.
    dp[i] = minimum squares for sum i
    """
    dp = [float('inf')] * (n + 1)
    dp[0] = 0
    
    # Precompute squares
    squares = []
    i = 1
    while i * i <= n:
        squares.append(i * i)
        i += 1
    
    for i in range(1, n + 1):
        for sq in squares:
            if sq > i:
                break
            dp[i] = min(dp[i], dp[i - sq] + 1)
    
    return dp[n]

# Test
for n in [12, 13, 1, 4]:
    print(f"numSquares({n}) = {num_squares(n)}")
# 12 = 4 + 4 + 4 (3 squares)
# 13 = 4 + 9 (2 squares)
# 1 = 1 (1 square)
# 4 = 4 (1 square)

***Figure 12.28:** Similar to coin change—try all perfect squares as "coins."*

- **Identify subproblems:** What decisions lead to optimal solution?
- **Define state:** What information do we need to track?
- **Write recurrence:** How do states relate to each other?
- **Base cases:** What are the simplest subproblems?
- **Optimize space:** Do we need the entire table or just recent rows?

---
## 11. Common Pitfalls


### Pitfall 1: Wrong Base Cases

**Listing 12.29 — Base Case Importance**

In [ ]:
# WRONG: Missing base case for coin change
def coin_change_wrong(coins, amount):
    dp = [float('inf')] * (amount + 1)
    # Missing: dp[0] = 0
    
    for i in range(1, amount + 1):
        for coin in coins:
            if coin <= i:
                dp[i] = min(dp[i], dp[i - coin] + 1)
    
    return dp[amount]

# RIGHT: Proper base case
def coin_change_right(coins, amount):
    dp = [float('inf')] * (amount + 1)
    dp[0] = 0  # 0 coins needed for amount 0
    
    for i in range(1, amount + 1):
        for coin in coins:
            if coin <= i and dp[i - coin] != float('inf'):
                dp[i] = min(dp[i], dp[i - coin] + 1)
    
    return dp[amount] if dp[amount] != float('inf') else -1

coins = [2]
amount = 4
print(f"Coins: {coins}, Amount: {amount}")
print(f"Wrong (no base): {coin_change_wrong(coins, amount)}")
print(f"Right: {coin_change_right(coins, amount)}")

***Figure 12.29:** Base cases anchor the recursion—without them, nothing works.*

### Pitfall 2: Off-by-One in Index

**Listing 12.30 — Index Mapping**

In [ ]:
# When dp array is size n+1, be careful with index mapping

def lcs_careful(text1, text2):
    m, n = len(text1), len(text2)
    # dp has size (m+1) x (n+1)
    # dp[i][j] corresponds to text1[:i] and text2[:j]
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            # Characters are at i-1 and j-1 in original strings!
            if text1[i - 1] == text2[j - 1]:  # NOT text1[i]
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])
    
    return dp[m][n]

text1, text2 = "abc", "ac"
print(f"LCS of '{text1}' and '{text2}': {lcs_careful(text1, text2)}")

print("\nRemember: dp[i][j] uses text1[i-1] and text2[j-1]")

***Figure 12.30:** When dp has extra row/column for base case, adjust string indices.*

### Pitfall 3: Wrong Iteration Order

**Listing 12.31 — Iteration Direction Matters**

In [ ]:
# For 0/1 knapsack (each item once): iterate BACKWARDS
# For unbounded knapsack (unlimited items): iterate FORWARDS

def knapsack_01_1d(weights, values, capacity):
    """0/1 Knapsack with 1D array - iterate backwards."""
    dp = [0] * (capacity + 1)
    
    for i in range(len(weights)):
        # BACKWARDS: don't use same item twice
        for w in range(capacity, weights[i] - 1, -1):
            dp[w] = max(dp[w], dp[w - weights[i]] + values[i])
    
    return dp[capacity]

def unbounded_knapsack_1d(weights, values, capacity):
    """Unbounded Knapsack with 1D array - iterate forwards."""
    dp = [0] * (capacity + 1)
    
    for w in range(1, capacity + 1):
        for i in range(len(weights)):
            if weights[i] <= w:
                # FORWARDS: allows reusing items
                dp[w] = max(dp[w], dp[w - weights[i]] + values[i])
    
    return dp[capacity]

weights, values = [2, 3], [3, 4]
capacity = 5

print(f"Weights: {weights}, Values: {values}, Capacity: {capacity}")
print(f"0/1 Knapsack: {knapsack_01_1d(weights, values, capacity)}")
print(f"Unbounded: {unbounded_knapsack_1d(weights, values, capacity)}")

***Figure 12.31:** Backwards iteration prevents reusing items in same round.*

### Pitfall 4: Counting vs Optimization

**Listing 12.32 — Combinations vs Permutations**

In [ ]:
# Counting COMBINATIONS (order doesn't matter): coins in outer loop
def coin_combinations(coins, amount):
    dp = [0] * (amount + 1)
    dp[0] = 1
    for coin in coins:           # Coin in outer loop
        for i in range(coin, amount + 1):
            dp[i] += dp[i - coin]
    return dp[amount]

# Counting PERMUTATIONS (order matters): amount in outer loop
def coin_permutations(coins, amount):
    dp = [0] * (amount + 1)
    dp[0] = 1
    for i in range(1, amount + 1):    # Amount in outer loop
        for coin in coins:
            if coin <= i:
                dp[i] += dp[i - coin]
    return dp[amount]

coins = [1, 2]
amount = 3

print(f"Coins: {coins}, Amount: {amount}")
print(f"Combinations (1+1+1, 1+2, 2+1 counted as same): {coin_combinations(coins, amount)}")
print(f"Permutations (1+1+1, 1+2, 2+1 all different): {coin_permutations(coins, amount)}")

***Figure 12.32:** Loop order determines whether we count combinations or permutations.*

### Pitfall 5: Integer Overflow

**Listing 12.33 — Handling Large Numbers**

In [ ]:
# Python handles big integers, but watch for problems asking modulo
MOD = 10**9 + 7

def unique_paths_mod(m, n):
    """Unique paths with modulo for large grids."""
    dp = [[1] * n for _ in range(m)]
    
    for i in range(1, m):
        for j in range(1, n):
            dp[i][j] = (dp[i - 1][j] + dp[i][j - 1]) % MOD
    
    return dp[m - 1][n - 1]

# Large grid
m, n = 100, 100
print(f"Unique paths in {m}x{n} grid:")
print(f"  Without mod: {unique_paths_mod.__code__.co_code}")  # Would be huge
print(f"  With mod 10^9+7: {unique_paths_mod(m, n)}")

# Always apply modulo at EACH step to prevent overflow
print("\nRule: (a + b) % MOD = ((a % MOD) + (b % MOD)) % MOD")

***Figure 12.33:** Apply modulo at each step to keep numbers manageable.*

---
# 📝 Exercises


### Exercise 1: Min Cost Climbing Stairs  (⭐ Easy)

Given an array cost where cost[i] is the cost of step i, find the minimum cost to reach the top. You can start from step 0 or 1, and can climb 1 or 2 steps at a time.

**Expected:** (Expected: [10,15,20] → 15)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - State Definition:**

`dp[i]` = minimum cost to reach step i.
- **Hint 2 - Recurrence:**

`dp[i] = min(dp[i-1], dp[i-2]) + cost[i]`
- **Hint 3 - Final Answer:**

                        To go past the top: `min(dp[n-1], dp[n-2])`
</details>

In [ ]:
# ✏️ [EX1]
# Min Cost Climbing Stairs - DP

def min_cost_climbing_stairs(cost):
    # Your code here
    pass

# Test your implementation (uncomment)
# print(min_cost_climbing_stairs([10, 15, 20]))  # 15
# print(min_cost_climbing_stairs([1, 100, 1, 1, 1, 100, 1, 1, 100, 1]))  # 6

### Exercise 2: House Robber II  (⭐⭐ Medium)

Houses are arranged in a circle (first and last are adjacent). You cannot rob adjacent houses. Find maximum amount you can rob.

**Expected:** (Expected: [2,3,2] → 3)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Circle Break:**

                        If we rob house 0, we cannot rob house n-1 (they're adjacent in circle).
- **Hint 2 - Two Cases:**

                        Case 1: Rob houses [0..n-2]. Case 2: Rob houses [1..n-1].
- **Hint 3 - Combine:**

`max(rob(nums[1:]), rob(nums[:-1]))` using linear house robber.
</details>

In [ ]:
# ✏️ [EX2]
# House Robber II - Circular DP

def rob_circular(nums):
    # Your code here
    pass

# Test your implementation (uncomment)
# print(rob_circular([2, 3, 2]))  # 3
# print(rob_circular([1, 2, 3, 1]))  # 4

### Exercise 3: Longest Increasing Path in Matrix  (⭐⭐⭐ Hard)

Find the longest increasing path in a matrix. You can move in 4 directions (up, down, left, right).

**Expected:** (Expected: Path 1→2→6→9 = length 4)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - DFS + Memo:**

                        Use DFS from each cell with memoization.
- **Hint 2 - State:**

`dp[i][j]` = longest path starting from cell (i,j).
- **Hint 3 - Recurrence:**

`dp[i][j] = 1 + max(dp[neighbor])` for all neighbors with larger value.
</details>

In [ ]:
# ✏️ [EX3]
# Longest Increasing Path in Matrix - DFS + Memoization

def longest_increasing_path(matrix):
    # Your code here
    pass

# Test your implementation (uncomment)
# matrix = [[9,9,4],[6,6,8],[2,1,1]]
# print(longest_increasing_path(matrix))  # 4, path: 1→2→6→9

### Exercise 4: Target Sum  (⭐⭐⭐ Hard)

Given array nums and target S, find number of ways to assign + or - to each number to achieve target sum.

**Expected:** (Expected: [1,1,1,1,1], target=3 → 5 ways)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Math Transform:**

                        P - N = target, P + N = sum → P = (target + sum) / 2
- **Hint 2 - Subset Sum:**

                        Count subsets with sum = P (0/1 knapsack counting variant).
- **Hint 3 - Edge Cases:**

                        Check if (target + sum) is even and non-negative.
</details>

In [ ]:
# ✏️ [EX4]
# Target Sum - Transform to Subset Sum

def find_target_sum_ways(nums, target):
    # Your code here
    pass

# Test your implementation (uncomment)
# print(find_target_sum_ways([1, 1, 1, 1, 1], 3))  # 5

### Exercise 5: Interleaving String  (⭐⭐⭐ Hard)

Given strings s1, s2, s3, check if s3 is formed by interleaving s1 and s2 (maintaining relative order).

**Expected:** (Expected: "aabcc" + "dbbca" → "aadbbcbcac" = True)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - State:**

`dp[i][j]` = True if s1[:i] and s2[:j] can form s3[:i+j].
- **Hint 2 - Recurrence:**

`dp[i][j] = (dp[i-1][j] and s1[i-1]==s3[i+j-1]) or (dp[i][j-1] and s2[j-1]==s3[i+j-1])`
- **Hint 3 - Base Case:**

`dp[0][0] = True`. Initialize first row and column.
</details>

In [ ]:
# ✏️ [EX5]
# Interleaving String - 2D DP

def is_interleave(s1, s2, s3):
    # Your code here
    pass

# Test your implementation (uncomment)
# print(is_interleave("aabcc", "dbbca", "aadbbcbcac"))  # True
# print(is_interleave("aabcc", "dbbca", "aadbbbaccc"))  # False

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Run all exercise cells** (make sure each one executed)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,12}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-12 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
_ORIGINAL_HASHES = {}

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you RAN all exercise cells first!

import json, re, os, urllib.request
import time as _time, datetime as _dt

WEEK = "Week_12"
URL  = "https://script.google.com/macros/s/AKfycbxepk2NvNg3Whad-WOPxdZI-mWnVJeNKCsZVspvk7Ku5YHC_oWv7376VrWLn_30nyI_vw/exec"

# ══════════════════════════════════════════════════════════
# SELF-HEALING TIMER — works even if Cell 1 was skipped
# ══════════════════════════════════════════════════════════
_TRACKER_FILE = os.path.join(
    '/content' if os.path.isdir('/content') else '/tmp',
    f'.cp2_session_{WEEK}.json'
)
IDLE_THRESHOLD = 300

def _sh_calc_active(heartbeats):
    """Calculate active seconds from a heartbeat list."""
    if len(heartbeats) < 2: return 0
    active = 0
    for i in range(1, len(heartbeats)):
        gap = heartbeats[i] - heartbeats[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _collect_timing():
    """
    Collect timing data from all available sources:
      1. In-memory variables (timer cell ran in this kernel)
      2. Persistent tracker file (previous sessions / kernel restarts)
      3. Minimal fallback (nothing available)
    """
    _info = {
        'active_time': 0, 'wall_time': 0, 'cells_run': 0,
        'heartbeat_count': 0, 'session_count': 0,
        'session_start': '', 'session_end': _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'source': 'none'
    }

    # ── Source 1: In-memory (timer cell ran this kernel) ──
    _has_memory = False
    try:
        _s = _SESSION_START        # noqa: F821
        _hb = list(_HEARTBEATS)    # noqa: F821
        _cr = _CELLS_RUN[0]        # noqa: F821
        _has_memory = True
        _info['active_time'] = _sh_calc_active(_hb)
        _info['wall_time'] = int(_time.time() - _s)
        _info['cells_run'] = _cr
        _info['heartbeat_count'] = len(_hb)
        _info['session_start'] = _SESSION_START_STR  # noqa: F821
        _info['source'] = 'memory'
    except NameError:
        pass

    # ── Source 2: Persistent file (adds previous sessions) ──
    try:
        with open(_TRACKER_FILE, 'r') as f:
            _t = json.load(f)
        sessions = _t.get('sessions', [])
        _info['session_count'] = len(sessions)

        if not _has_memory and sessions:
            # Timer cell was NOT run — reconstruct from file
            last = sessions[-1]
            hb = last.get('heartbeats', [])
            _info['active_time'] = _sh_calc_active(hb)
            _info['wall_time'] = int(hb[-1] - hb[0]) if len(hb) >= 2 else 0
            _info['cells_run'] = last.get('cells_run', len(hb))
            _info['heartbeat_count'] = len(hb)
            _info['session_start'] = last.get('start', '')
            _info['source'] = 'file-current'

        # Add previous sessions' active time
        if len(sessions) > 1:
            prev_active = 0
            prev_cells = 0
            prev_hb = 0
            cutoff = -1 if _has_memory else -1  # exclude current session
            for sess in sessions[:cutoff]:
                hb = sess.get('heartbeats', [])
                prev_active += _sh_calc_active(hb)
                prev_cells += sess.get('cells_run', len(hb))
                prev_hb += len(hb)
            _info['prev_active_time'] = prev_active
            _info['prev_cells_run'] = prev_cells
            _info['prev_heartbeats'] = prev_hb
            _info['total_active_time'] = _info['active_time'] + prev_active
            _info['total_cells_run'] = _info['cells_run'] + prev_cells
            if _info['source'] == 'none':
                _info['source'] = 'file-prev'

    except (FileNotFoundError, json.JSONDecodeError, KeyError, ValueError):
        pass

    # Ensure totals exist
    _info.setdefault('total_active_time', _info['active_time'])
    _info.setdefault('total_cells_run', _info['cells_run'])
    return _info

# ── Persist final heartbeat snapshot before collecting ──
try:
    _t = json.load(open(_TRACKER_FILE))
    if _t['sessions']:
        _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
        _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
        json.dump(_t, open(_TRACKER_FILE, 'w'))
except Exception:
    pass

_timing = _collect_timing()

# ── Display timing ──
if _timing['source'] == 'none':
    print("⚠️  Zamanlayıcı verisi bulunamadı — ilk hücreyi çalıştırmayı unutmayın.")
    print("    Gönderim yine de yapılacak (süre: 0 olarak kaydedilir).")
else:
    _a = _timing['active_time']
    _w = _timing['wall_time']
    print(f"⏱️  Bu oturum — Aktif: {_a//60}m {_a%60}s  |  Duvar: {_w//60}m {_w%60}s")
    print(f"🔢  Hücreler: {_timing['cells_run']}  |  Heartbeat: {_timing['heartbeat_count']}")
    if _timing['session_count'] > 1:
        _ta = _timing['total_active_time']
        print(f"📂  Toplam {_timing['session_count']} oturum — Toplam aktif: {_ta//60}m {_ta%60}s  |  Toplam hücre: {_timing['total_cells_run']}")
    if _timing['total_active_time'] < 120:
        print("⚠️  Toplam aktif süre < 2 dk — çalıştırdığınız hücre sayısı düşük olabilir.")

_time_on_page = _timing['total_active_time']

# ══════════════════════════════════════════════════════════
# STUDENT INFO CHECK
# ══════════════════════════════════════════════════════════
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın (öğrenci bilgileri).")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın — bazı alanlar boş.")

# ══════════════════════════════════════════════════════════
# EXERCISE ANSWER COLLECTION (improved with retry)
# ══════════════════════════════════════════════════════════
_EX_PATTERN = re.compile(r'#\s*✏️\s*\[EX(\d+)\]')

def _extract_exercise(src):
    """Try to extract exercise ID and code from a source string."""
    m = _EX_PATTERN.match(src)
    if not m: return None, None
    ex_id = 'ex' + m.group(1)
    clean = '\n'.join(src.split('\n')[1:]).strip()
    return ex_id, clean

_answers = {}

# ── Method 1: In[] list (most reliable for Run-All) ──
try:
    for _src in In:
        if not _src: continue
        _eid, _code = _extract_exercise(_src)
        if _eid:
            _answers[_eid] = {
                'code': _code,
                'modified': len(_code) > 5
            }
except NameError:
    pass

# ── Method 2: IPython history_manager (flush first!) ──
if not _answers:
    try:
        _ipy = get_ipython()
        # Force flush so Run-All history is fully written
        try: _ipy.history_manager.db.commit()
        except Exception: pass
        _time.sleep(0.5)  # small delay for DB sync
        for _sess, _line, _src in _ipy.history_manager.get_range(output=False):
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': len(_code) > 5
                }
    except Exception:
        pass

# ── Method 3: Read notebook file (VS Code / local Jupyter) ──
if not _answers:
    _nb_path = None
    try:
        _nb_path = __vsc_ipynb_file__
    except NameError:
        _candidates = [f for f in os.listdir('.') if f.endswith('.ipynb') and WEEK in f]
        if len(_candidates) == 1:
            _nb_path = _candidates[0]
    if _nb_path and os.path.exists(str(_nb_path)):
        with open(str(_nb_path), 'r', encoding='utf-8') as _f:
            _nb = json.load(_f)
        for _cell in _nb['cells']:
            if _cell['cell_type'] != 'code': continue
            _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': len(_code) > 5
                }

# ── Method 4: Colab notebook file at /content/ ──
if not _answers:
    try:
        _colab_candidates = [f for f in os.listdir('/content') if f.endswith('.ipynb') and WEEK in f]
        if _colab_candidates:
            _nb_path = os.path.join('/content', _colab_candidates[0])
            with open(_nb_path, 'r', encoding='utf-8') as _f:
                _nb = json.load(_f)
            for _cell in _nb['cells']:
                if _cell['cell_type'] != 'code': continue
                _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
                _eid, _code = _extract_exercise(_src)
                if _eid:
                    _answers[_eid] = {
                        'code': _code,
                        'modified': len(_code) > 5
                    }
    except Exception:
        pass

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys(), key=lambda x: int(x[2:]) if x[2:].isdigit() else 999))}")

if not _answers:
    print("\n⚠️  Hiçbir exercise yanıtı bulunamadı!")
    print("Submit'ten ÖNCE tüm exercise hücrelerini çalıştırdığınızdan emin olun.")
    print("\n💡 İpucu: 'Run All' yerine exercise hücrelerini tek tek çalıştırıp")
    print("   ardından bu hücreyi çalıştırmayı deneyin.")
    raise SystemExit()

# ══════════════════════════════════════════════════════════
# SEND SUBMISSION
# ══════════════════════════════════════════════════════════
_data = json.dumps({
    'week': WEEK,
    'studentId': _sid,
    'studentName': _sname,
    'studentEmail': _semail,
    'classCode': _scode,
    'source': 'dsa-notebook',
    'timeOnPage': _time_on_page,
    'wallTime': _timing['wall_time'],
    'cellsRun': _timing['total_cells_run'],
    'sessionCount': _timing['session_count'],
    'sessionStart': _timing['session_start'],
    'sessionEnd': _timing['session_end'],
    'timerSource': _timing['source'],
    'answers': _answers
}).encode('utf-8')

print("📡 Gönderiliyor...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get('success'):
        print(f"\n✅ {_result['message']}")
        print('📧 Onay için e-postanızı kontrol edin.')
    else:
        print(f"\n❌ {_result.get('message', 'Gönderim başarısız')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  İstek gönderildi — onay için e-postanızı kontrol edin.")
    print(f"(E-posta gelmezse tekrar deneyin veya hocaya başvurun)")
